In [1]:
import sys, time, pickle
import torch

sys.path.insert(0, '../../Utils')
from global_constants import gpu_device

sys.path.insert(0, '../')
from BERT_models import BERT_QNLI_MODEL

from _utils import sample_random_glue_qnli, get_continuation_mapping, \
                    get_continuous_attributions, get_continuous_raw_inputs, \
                    collect_info_for_metric, save_info, download_HTML

In [2]:
qnli_data_raw, targets, idxs = sample_random_glue_qnli()

Reusing dataset glue (/home/user/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-e6d0a9c9270b6822.arrow
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-c46c5e78d2bda808.arrow


In [3]:
model = BERT_QNLI_MODEL()
tokenizer = model.tokenizer

In [4]:
#define some containers to save some info
model_out_list, raw_attr_list, conti_attr_list, raw_input_list = [], [], [], []

In [5]:
from captum.attr import GuidedBackprop
from captum.attr import visualization 

In [6]:
gbp  = GuidedBackprop(model)

In [7]:
def generate_record(raw_datum, target): #raw_datum expected to be a tuple/list of 2 sentences
    #tokenizer operations
    tokenized = tokenizer(raw_datum, truncation=True, return_offsets_mapping=True)
    offset_mappings = tokenized['offset_mapping']
    #concatenate the two offset_mappings together because they are fed in together
    conti_map = get_continuation_mapping(offset_mappings[0]) + get_continuation_mapping(offset_mappings[1])
    #change the first input_id of the second sentence to be the last input_id of the 1st sentence (i.e. an [END] token))
    tokenized_input_ids = tokenized['input_ids'][0] + \
                        [tokenized['input_ids'][1][i] if i != 0 else tokenized['input_ids'][0][-1] \
                         for i in range(len(tokenized['input_ids'][1]))]
    input_ids = torch.tensor(tokenized_input_ids).unsqueeze(0).to(gpu_device)
    detokenized = [t.replace('#', '') for t in tokenizer.convert_ids_to_tokens(input_ids[0])]
    
    #feeding input forward 
    input_emb = model.get_embeddings(input_ids)
    pred_prob = model(input_emb).item()

     #categorizing results
    pred_class = 'No Entailment' if pred_prob < 0.5 else 'Entailment' 
    true_class = 'No Entailment' if target < 0.5 else 'Entailment' 
    
    #attribution algorithm working
    attribution = gbp.attribute(input_emb)
    word_attributions = attribution.squeeze(0).sum(dim=1)
#     word_attributions = attr_normalizing_func(word_attributions)
    word_attributions /= torch.norm(word_attributions)
    attr_score = torch.sum(word_attributions)
    attr_class = 'No Entailment' if attr_score < 0.5 else 'Entailment'
    convergence_score = None
    
    
#     #re-organizing tensors and arrays because words get split down
    conti_attr = get_continuous_attributions(conti_map, word_attributions)
    raw_input = get_continuous_raw_inputs(conti_map, detokenized)

#     print(f'word attributions {word_attributions}')
    print(f'pred_prob {pred_prob}')
#     print(f'pred_class {pred_class}')
#     print(f'true_class {true_class}')
#     print(f'attribution {attribution}')
#     print(f'attr_class {attr_class}')
#     print(f'attr_score {attr_score}')
#     print(f'raw_input {raw_input}')

        
# #     collect info for metrics later
    collect_info_for_metric(model_out_list, pred_prob, raw_attr_list, attribution, conti_attr_list, conti_attr, raw_input_list, raw_input)
        
    
    visual_record = visualization.VisualizationDataRecord(word_attributions=conti_attr,
                                                         pred_prob=pred_prob,
                                                         pred_class=pred_class,
                                                         true_class=true_class,
                                                         attr_class=attr_class,
                                                         attr_score=attr_score,
                                                         raw_input_ids=raw_input,
                                                         convergence_score=convergence_score)
        
        
    return visual_record
      
    

In [8]:
for i, (datum_raw, target) in enumerate(zip(qnli_data_raw, targets), start=1):
#     example_1 = "How many times has the South Florida/Miami area hosted the Super Bowl?"
#     example_2 = "The South Florida/Miami area has previously hosted the event 10 times (tied for most with New Orleans), with the most recent one being Super Bowl XLIV in 2010."
    
#     example_1 = "When did the third Digimon series begin?"
#     example_2 = "Unlike the two seasons before it and most of the seasons that followed, Digimon Tamers takes a darker and more realistic approach to its story featuring Digimon who do not reincarnate after their deaths and more complex character development in the original Japanese."
#     datum_raw, target = [example_1, example_2], 1
    print(f'Raw datum: {datum_raw}') #datum expected to be a list of 2 sentences
    print(f'GT target: {target}')
    visual_record=generate_record(datum_raw, target)
    print(visualization.visualize_text([visual_record])) 


Raw datum: ['What would a teacher assess the levels of a student on?', 'For example, an experienced teacher and parent described the place of a teacher in learning as follows: "The real bulk of learning takes place in self-study and problem solving with a lot of feedback around that loop.']
GT target: 0
word attr tensor([ 0.4524,  0.0937,  0.0895,  0.0985,  0.0676,  0.1496,  0.0570,  0.0049,
         0.0922,  0.0396,  0.0612, -0.0501,  0.2952, -0.0819,  0.0579,  0.1388,
         0.0226,  0.1011,  0.0542,  0.1844,  0.0013,  0.0429,  0.0388,  0.1257,
         0.1153, -0.1831,  0.1112,  0.0133,  0.0654,  0.1097,  0.0092,  0.0690,
         0.2735,  0.1869,  0.1536, -0.0150,  0.0334,  0.1812, -0.1302,  0.0233,
         0.0417,  0.0133,  0.0272,  0.1617,  0.0398,  0.1582,  0.0501,  0.1367,
         0.0465,  0.0651,  0.0852,  0.1413,  0.1050, -0.0746,  0.2528,  0.0276,
         0.2055,  0.0007, -0.1995], device='cuda:0')
conti attr [tensor(0.4524, device='cuda:0'), tensor(0.0937, device='cuda

/home/user/anaconda3/envs/metric/lib/python3.9/site-packages/captum/attr/_core/guided_backprop_deconvnet.py:64: UserWarning: Setting backward hooks on ReLU activations.The hooks will be removed after the attribution is finished
  warnings.warn(


<IPython.core.display.HTML object>
Raw datum: ['What company created Doctor Who?', 'Who character by BBC Television in the early 1960s, a myriad of stories have been published about Doctor Who, in different media: apart from the actual television episodes that continue to be produced by the BBC, there have also been novels, comics, short stories, audio books, radio plays, interactive video games, game books, webcasts, DVD extras, and even stage performances.']
GT target: 1
word attr tensor([ 3.3732e-01, -2.2059e-01,  1.4004e-01,  7.9389e-02,  3.8219e-01,
        -4.2421e-02, -6.5379e-01,  6.1251e-02, -1.6984e-01, -8.9809e-02,
         1.0184e-01,  5.8344e-02,  1.3749e-01,  8.6075e-02,  2.8749e-02,
         4.1341e-03,  3.2042e-02,  1.7134e-02, -2.5181e-02, -2.1430e-02,
        -1.4146e-02, -6.7291e-03, -3.3449e-02,  1.2745e-02,  1.3772e-02,
        -2.6559e-02, -1.0639e-02,  1.6971e-01, -1.9852e-02,  1.2852e-02,
        -2.6407e-02, -1.3851e-02, -3.5502e-02,  1.1969e-01, -3.9474e-02,
 

<IPython.core.display.HTML object>
Raw datum: ['What was the name of the Media Day event for Super Bowl 50?', "The game's media day, which was typically held on the Tuesday afternoon prior to the game, was moved to the Monday evening and re-branded as Super Bowl Opening Night."]
GT target: 1
word attr tensor([-0.2485, -0.1521,  0.2237, -0.0537,  0.1408, -0.1003, -0.0191,  0.2411,
         0.1431, -0.0523,  0.1088,  0.1575,  0.1088, -0.2578, -0.1163,  0.0881,
         0.2660, -0.0327,  0.0227,  0.0171, -0.0884,  0.3716,  0.1319,  0.0669,
        -0.0849,  0.0175,  0.1364,  0.0548,  0.0369, -0.0061,  0.1682,  0.0870,
         0.2130, -0.0164, -0.0443, -0.0118,  0.0808,  0.1720,  0.1020, -0.0700,
         0.0160, -0.0087,  0.0175, -0.0286,  0.1829,  0.0426,  0.1092, -0.2295,
         0.1700,  0.2042,  0.1077,  0.1176,  0.0529, -0.0903], device='cuda:0')
conti attr [tensor(-0.2485, device='cuda:0'), tensor(-0.1521, device='cuda:0'), tensor(0.2237, device='cuda:0'), tensor(-0.0537, device='

<IPython.core.display.HTML object>
Raw datum: ['How many Doctor Who soundtracks have been released since 2005?', 'The fourth was released on 4 October 2010 as a two disc special edition and contained music from the 2008–2010 specials (The Next Doctor to End of Time Part 2).']
GT target: 0
word attr tensor([ 0.6671, -0.0352,  0.0564,  0.0017,  0.0243,  0.1744,  0.0414,  0.1263,
         0.0844, -0.0129, -0.0633,  0.0568, -0.1697, -0.2617,  0.4277,  0.0908,
         0.0533, -0.0086, -0.0054,  0.0533,  0.0940,  0.0155,  0.0497,  0.0178,
         0.0585,  0.0118,  0.0269,  0.1466,  0.0773,  0.0935,  0.0821,  0.0467,
         0.0466,  0.0534,  0.0463,  0.0866,  0.1825,  0.0828,  0.0183,  0.1175,
         0.0058,  0.0759,  0.0997,  0.0242,  0.0363,  0.0138,  0.0746,  0.0831,
         0.1748, -0.0451], device='cuda:0')
conti attr [tensor(0.6671, device='cuda:0'), tensor(-0.0352, device='cuda:0'), tensor(0.0564, device='cuda:0'), tensor(0.0017, device='cuda:0'), tensor(0.0243, device='cuda:0')

...

<IPython.core.display.HTML object>
Raw datum: ['Who did Britain exploit in India?', 'Although a substantial number of colonies had been designed to provide economic profit and to ship resources to home ports in the seventeenth and eighteenth centuries, Fieldhouse suggests that in the nineteenth and twentieth centuries in places such as Africa and Asia, this idea is not necessarily valid:']
GT target: 0
word attr tensor([ 0.3519,  0.0745, -0.0775,  0.3259, -0.0366, -0.1291, -0.0994, -0.0436,
        -0.1283, -0.1183, -0.0482, -0.0684,  0.0076,  0.0119,  0.0212, -0.0559,
         0.0462,  0.0230, -0.0299, -0.0217, -0.0311,  0.1219,  0.0809,  0.0075,
         0.0706,  0.0847, -0.0145, -0.0135,  0.1287,  0.0343,  0.1333,  0.1131,
        -0.0012,  0.0832,  0.0837,  0.1749,  0.1162,  0.0328, -0.0422,  0.3522,
        -0.0848,  0.0842,  0.0243,  0.0835,  0.0630,  0.0494,  0.0734,  0.1096,
         0.1147,  0.0189, -0.0415,  0.1480,  0.0086,  0.2223,  0.1475, -0.1007,
        -0.0178, -0.0568

<IPython.core.display.HTML object>
Raw datum: ['What popular environmentalist is also a university alumni member?', 'In science, alumni include astronomers Carl Sagan, a prominent contributor to the scientific research of extraterrestrial life, and Edwin Hubble, known for "Hubble\'s Law", NASA astronaut John M. Grunsfeld, geneticist James Watson, best known as one of the co-discoverers of the structure of DNA, experimental physicist Luis Alvarez, popular environmentalist David Suzuki, balloonist Jeannette Piccard, biologists Ernest Everett']
GT target: 1
word attr tensor([-1.8149e-01, -3.6558e-02, -5.4785e-02, -8.9095e-02,  1.0598e-01,
        -1.9518e-02, -1.4297e-01, -1.6503e-02, -1.4117e-02, -8.0978e-03,
         2.5182e-02, -2.6158e-01,  2.8511e-01,  2.9986e-01, -5.8321e-02,
         3.0019e-02, -1.6300e-02,  4.5490e-02,  2.8518e-02, -2.2679e-01,
         1.9004e-02, -5.1447e-02, -1.9281e-02, -7.7909e-04, -2.3034e-03,
         1.4637e-02,  5.8443e-02, -1.9790e-02,  4.6305e-02, -4.7

<IPython.core.display.HTML object>
Raw datum: ['What publication did Philip Howard work for?', 'Responding to the findings of the survey in The Times newspaper, journalist Philip Howard maintained that, "to compare the violence of Dr Who, sired by a horse-laugh out of a nightmare, with the more realistic violence of other television series, where actors who look like human beings bleed paint that looks like blood, is like comparing Monopoly with the property market in London: both are fantasies, but one is meant to be taken seriously."']
GT target: 1
word attr tensor([-2.6737e-01,  2.8919e-02, -1.9882e-01,  7.0749e-02,  4.6940e-02,
         2.6842e-01,  1.6274e-02, -1.2432e-01, -1.0279e-01,  2.0648e-01,
         3.6576e-02, -9.8034e-02, -6.4064e-02, -7.4530e-02, -2.0915e-01,
         1.5743e-02, -8.6402e-02, -1.6947e-03,  2.2469e-01, -1.1034e-01,
         3.5698e-02,  4.7201e-01, -6.0027e-02, -3.6128e-02, -1.5647e-02,
         2.2575e-01,  2.1677e-02, -3.8282e-03,  5.4521e-02, -8.0470e

<IPython.core.display.HTML object>
Raw datum: ['If the apparant force of two fermions is attractive, what is the spin function?', 'If two identical fermions (e.g. electrons) have a symmetric spin function (e.g. parallel spins) the spatial variables must be antisymmetric (i.e. they exclude each other from their places much as if there was a repulsive force), and vice versa, i.e. for antiparallel spins the position variables must be symmetric (i.e. the apparent force must be attractive).']
GT target: 1
word attr tensor([-0.1783, -0.0828, -0.0418, -0.0557,  0.0304, -0.0346,  0.0512, -0.1131,
        -0.0938, -0.0690, -0.0121, -0.0497, -0.0391,  0.1921, -0.1345, -0.2241,
         0.0228, -0.1010, -0.0375,  0.0116, -0.1435,  0.1757,  0.0919, -0.1009,
        -0.0787, -0.0799,  0.0038, -0.0209, -0.0210,  0.0101, -0.0385, -0.0122,
        -0.0343,  0.0394,  0.0296,  0.0317, -0.0792, -0.0144,  0.0976, -0.1139,
        -0.0116, -0.0161, -0.0115, -0.0893,  0.0508, -0.0498,  0.0877,  0.0773,
    

<IPython.core.display.HTML object>


In [9]:
file_name_base = 'Guided_Backprop'

In [10]:
save_info(idxs, qnli_data_raw, targets, model_out_list, raw_attr_list, conti_attr_list, raw_input_list, 
          fname=f'../MethodOutputs/{file_name_base}_out.pkl')

{'indices': [4013,
  2281,
  2771,
  5109,
  1852,
  2016,
  642,
  3912,
  4427,
  2330,
  4581,
  2611,
  5186,
  2700,
  4920,
  4335,
  1246,
  2811,
  5290,
  5385,
  4774,
  653,
  2816,
  916,
  3297,
  3459,
  1475,
  1287,
  4548,
  1476,
  2328,
  2208,
  2772,
  5301,
  869,
  4968,
  4456,
  4711,
  1929,
  634,
  3774,
  5389,
  497,
  4901,
  4602,
  831,
  1292,
  1087,
  3159,
  3492],
 'raw_data': [['What would a teacher assess the levels of a student on?',
   'For example, an experienced teacher and parent described the place of a teacher in learning as follows: "The real bulk of learning takes place in self-study and problem solving with a lot of feedback around that loop.'],
  ['What company created Doctor Who?',
   'Who character by BBC Television in the early 1960s, a myriad of stories have been published about Doctor Who, in different media: apart from the actual television episodes that continue to be produced by the BBC, there have also been novels, comics, sho

In [11]:
download_HTML(cur_file_name=f'{file_name_base}_QNLI_BERT.ipynb',
              out_file_name=f'{file_name_base}_QNLI_BERT.html')